In [5]:
import pandas as pd
import numpy as np
import duckdb
import os
import sys
import sys
sys.path.insert(0, os.path.abspath("../.."))
from utils import time, data_processor, constants

In [6]:
all_countries = set(constants.eu + constants.americas + constants.asia + constants.africa + constants.oceania)
eu_27_countries = constants.eu
balkans = constants.balkans

In [7]:
    # SELECT 
    #     bvd_id_number,
    #     operating_revenue_turnover_,
    #     number_of_employees,
    #     costs_of_goods_sold,
    #     material_costs,
    #     added_value,
    #     total_assets,
    #     profit_margin_,
    #     operating_revenue_turnover_,
    #     EXTRACT(YEAR FROM closing_date) AS year  

In [ ]:
listed_financials_detail_path = "..\\..\\data_raw\\listed_financials\\detailed_format_industries\\"
# listed_financials_detail_path = "..\\..\\data_raw\\listed_financials\\segment_data\\"

def fetch_data_for_country(country_iso_code):
    query = f"""
    SELECT
        bvd_id_number,
        fixed_assets, #
        net_assets,
        share_capital, 
        total_assets, #
        total_current_assets, 
        total_current_liabilities,
        non_current_liabilities, 
        number_of_employees as number_of_employees_lf,
        total_revenues, #
        enterprise_value,
        EXTRACT(YEAR FROM closing_date) AS year  
    FROM 
        parquet_scan('{listed_financials_detail_path}*.parquet')
    WHERE
        CAST(bvd_id_number AS STRING) LIKE '{country_iso_code}%'
    ORDER BY 
        bvd_id_number, year
    """

    conn = duckdb.connect()
    df = conn.execute(query).fetchdf()

    return df


In [17]:
import os

output_path = "..\\..\\data_processed\\listed_financials_processed\\"

# Ensure the directory exists
os.makedirs(output_path, exist_ok=True)

for country in ['IT']:
    df = fetch_data_for_country(country)
    df = df.loc[df['year']>=2007]
    df.to_parquet(f"{output_path}listed_financials_{country}.parquet")
    print(f"Saved: {output_path}listed_financials_{country}.parquet")
    # print number of rows
    print(f"Number of rows: {len(df)}")


BinderException: Binder Error: Referenced column "fixed_assets" not found in FROM clause!
Candidate bindings: "parquet_scan.geo_segment_sales", "parquet_scan.geo_segment_assets", "parquet_scan.business_lines_assets", "parquet_scan.business_lines_footnote_cap_exp_", "parquet_scan.business_lines_footnote_insurance"
LINE 4:         fixed_assets,
                ^

In [11]:
df

,bvd_id_number,fixed_assets,net_assets,share_capital,total_assets,total_current_assets,total_current_liabilities,non_current_liabilities,number_of_employees_lf,total_revenues,enterprise_value,year
0,IT00047400320,4.104097e+08,9.125070e+07,6734597.96,7.535938e+08,3.431841e+08,4.395415e+08,2.228016e+08,7650.0,1.207197e+09,NaN,2000
1,IT00047400320,5.073910e+08,9.267044e+07,6751124.58,9.905473e+08,4.831563e+08,5.492354e+08,3.486415e+08,7048.0,1.219027e+09,NaN,2001
2,IT00047400320,4.915230e+08,8.217000e+07,6561000.00,9.342930e+08,4.427700e+08,6.124140e+08,2.397090e+08,9335.0,1.636122e+09,NaN,2002
3,IT00047400320,3.979130e+08,2.973400e+07,6581000.00,8.387250e+08,4.408120e+08,5.764470e+08,2.325440e+08,8797.0,1.496740e+09,NaN,2003
4,IT00047400320,3.108240e+08,-8.725400e+07,13267000.00,7.844760e+08,4.736520e+08,6.460420e+08,2.256880e+08,NaN,1.303867e+09,NaN,2004
...,...,...,...,...,...,...,...,...,...,...,...,...
7720,IT97579210010,6.648520e+08,7.473370e+08,4863000.00,1.506568e+09,8.417160e+08,5.470660e+08,2.835450e+08,9059.0,1.269596e+09,3.263691e+09,2020
7721,IT97579210010,8.624280e+08,9.454550e+08,4863000.00,1.847020e+09,9.845920e+08,6.864850e+08,3.446400e+08,10579.0,1.501434e+09,6.363820e+09,2021
7722,IT97579210010,1.070571e+09,1.084696e+09,4863000.00,2.228150e+09,1.157579e+09,8.682600e+08,3.880210e+08,13467.0,1.910566e+09,3.808954e+09,2022
7723,IT97579210010,1.046457e+09,1.203246e+09,4863000.00,2.373980e+09,1.327523e+09,8.914150e+08,3.658420e+08,14798.0,2.156302e+09,2.363844e+09,2023
